<a href="https://colab.research.google.com/github/vjmaurina/Projeto_Imersao_Avaliacao/blob/main/Projeto_Avaliacao_Alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q google-generativeai

In [ ]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

model_embedContent = "models/embedding-001"
generation_config = {"temperature": 0,"candidate_count": 1}

In [ ]:
# Carregando dados do CSV
df = pd.read_csv('https://raw.githubusercontent.com/vjmaurina/Projeto_Imersao_Avaliacao/main/dataset_reg10.csv')

# Convertendo o DataFrame em uma lista de dicionários
documents = df.to_dict('records')

In [ ]:
df = pd.DataFrame(documents)

In [ ]:
df

In [ ]:
def embed_fn(title, text):
    return genai.embed_content(model=model_embedContent,
                               content=text,
                               title=title,
                               task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [ ]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["ID_Pedido"], row["Data_Pedido"]), axis=1)
df

In [ ]:
def gerar_e_buscar_consulta(consulta, base, model):
    embedding_da_consulta = genai.embed_content(model=model,
                                               content=consulta,
                                               task_type="RETRIEVAL_QUERY")["embedding"]
    produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)
    indice = np.argmax(produtos_escalares)
    return df.iloc[indice]["Data_Pedido"]

In [ ]:
consulta = "Qual é a Data do Pedido US-2016-108966?"
trecho = gerar_e_buscar_consulta(consulta, df, model_embedContent)
prompt = f"Reescreva esse texto apenas informando somente **Data do Pedido é:**: {trecho}"
model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)